# Testing SVHN Localization Accuracy

## Importing Libraries

In [1]:
from collections import namedtuple
import random
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils

## Defining a Function to Grab Ground Truth Bounding Boxes

In [2]:
def get_gt_contours(i,digitStruct):
    gt_contours = []
    boxes = digitStruct.loc[f'{str(i)}.png', 'boxes']
    for box in boxes:
        x = box['left']
        y = box['top']
        w = box['width']
        h = box['height']
        gt_contours.append([x, y, x+w, y+h])
    return gt_contours

## Defining a Function to Calculate Intersection Over Uinion Between 2 Boxes

In [3]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

## Defining a Function to Remove Dublicated Contours

In [4]:
def removeDuplicateContours(pred_contours):
    # Corner Case: Image has only 1 digit and 1 contour predicted 
    if(len(pred_contours)==1):
        return pred_contours
    
    
    filtered_contours = []
    #pred_contours = [list(x) for x in set([tuple(L) for L in pred_contours])]
    i=0
    while(i<len(pred_contours)):
        filtered_contours.append(pred_contours[i])
        # Corner Case: Last two contours are not duplicates
        if i == len(pred_contours)-1:
            break    
        j=i+1
        while(j<len(pred_contours)):
            iou = bb_intersection_over_union(pred_contours[i], pred_contours[j])
            if iou>0.8:
                pred_contours.pop(j)  
            j+=1
        i+=1     
    return filtered_contours

## Defining a Function to Test Accuracy

In [6]:
def test_accuracy(dataset,images_number):
    digitStruct = pd.read_json(f'data/{dataset}/digitStruct.json')
    digitStruct.set_index('filename', inplace=True)
    tot_p = 0
    tot_r = 0
    tot_f1 = 0
    # clahe = cv2.createCLAHE(clipLimit=1, tileGridSize=(5,5))
    for i in range(1,images_number+1):
        img = cv2.imread(f'data/{dataset}/{i}.png')
        height = img.shape[0]
        width = img.shape[1]
        bfilter= cv2.bilateralFilter(img,11,17,17)
        gray = cv2.cvtColor(bfilter,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray,30,150)
        keypoints= cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours=imutils.grab_contours(keypoints)
        edged_cont = cv2.drawContours(img.copy(), contours, -1, (0,255,0), 1)
        sorted_contours=sorted(contours,key=cv2.contourArea,reverse=True)

        pred_contours = []
        for contour in sorted_contours:
            [x, y, w, h] = cv2.boundingRect(contour)
            if ( (h*w > 0.015*height*width) and (y<0.5*height) and (0.25*width< x <0.75*width)):
                pred_contours.append([x, y, x+w, y+h])

        # pred_contours = [list(x) for x in set([tuple(L) for L in pred_contours])]
        filtered_contours = removeDuplicateContours(pred_contours)

        gt_contours = []
        gt_contours = get_gt_contours(i,digitStruct)

        match_contours = []
        for contour in filtered_contours:
            for gt_contour in gt_contours:
                iou = bb_intersection_over_union(contour, gt_contour)
                if iou > 0.4:
                    match_contours.append({'box': contour, 'iou':iou})

        if len(match_contours) > len(gt_contours):
            match_contours = sorted(match_contours, key=lambda c: c['iou'], reverse=True)
            match_contours = match_contours[:len(gt_contours)]


        true_pos = len(match_contours)
        false_pos = len(filtered_contours) - len(match_contours)
        false_neg = len(gt_contours) - len(match_contours)

        if (true_pos + false_pos)==0:
            precision = 0
        else:
            precision = true_pos / (true_pos + false_pos)

        recall = true_pos / (true_pos + false_neg)

        if true_pos == 0:
            f1_score = 0
        else:
            f1_score = (2*precision*recall) / (precision+recall)

        tot_p += precision
        tot_r += recall
        tot_f1 += f1_score



    tot_p/=images_number
    tot_r/=images_number
    tot_f1/=images_number   


    print(f"Total Precision: {tot_p*100:0.3f}%")
    print(f"Total Recall: {tot_r*100:0.3f}%")
    print(f"Total F1-Score: {tot_f1*100:0.3f}%")

## Testing Accuracy on Training Dataset

In [7]:
test_accuracy('train',33402)

Total Precision: 49.946%
Total Recall: 74.814%
Total F1-Score: 56.907%


## Testing Accuracy on Testing Dataset

In [8]:
test_accuracy('test',13068)

Total Precision: 47.490%
Total Recall: 62.636%
Total F1-Score: 51.164%
